In [3]:
import os
import json
import pandas as pd
import traceback


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY = os.getenv("GOOGLE_API_KEY")

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=KEY)

In [7]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002A31638BA30>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002A31640F340>, default_metadata=())

In [8]:
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [10]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given above the text, it is your job to \
create a quiz of {number} multiple questions for {subject} students in {tone} tone.
Make sure to format your response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject","tone","response_json"],
    template=TEMPLATE
)

In [12]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt,output_key="quiz",verbose=True)

d:\Projects\MCQ_Generator\env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
TEMPLATE2 = """
You are an expert grammerian and writer. Given a multiple choice question for {subject} students.\
you need to evaluate the complexicity of the question and give a complete analysis of the quiz. only use at max 50 words for complexicity
if the quiz is not at par with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be changed and the change the tone such that it perfectly fit the students ability.\
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [15]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [16]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain,review_chain],
    input_variables = ["text", "number","subject","tone","response_json"],
    output_variables = ["quiz","review"],
    verbose = True
)

In [33]:
file_path = 'D:\Projects\MCQ_Generator\data.txt'

In [34]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [35]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [36]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [37]:
NUMBER = 5
SUBJECT = "electronic engineering"
TONE = "tough"

In [38]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [39]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [40]:
print(type(response))

<class 'dict'>


In [41]:
quiz = response.get("quiz")

In [42]:
quiz_string = quiz.replace("```json", "").replace("```", "").strip()

In [43]:
print(quiz_string)

{
  "1": {
    "mcq": "A critical design consideration for transmitters, directly impacting signal strength and susceptibility to noise, is:",
    "options": {
      "a": "Modulation type",
      "b": "Carrier wave frequency", 
      "c": "Power consumption",
      "d": "Antenna size"
    },
    "correct": "c" 
  },
  "2": {
    "mcq": "Which statement regarding instrumentation engineering is MOST accurate?",
    "options": {
      "a": "It primarily focuses on theoretical physics principles.",
      "b": "It designs devices exclusively for standalone measurement tasks.",
      "c": "It often utilizes principles like the Doppler effect for measurement.", 
      "d": "It has minimal overlap with control engineering concepts."
    },
    "correct": "c" 
  },
  "3": {
    "mcq": "The evolution of electronics engineering was significantly propelled by the invention of the:",
    "options": {
      "a": "Capacitor",
      "b": "Resistor",
      "c": "Transistor", 
      "d": "Mechanical swi

In [44]:
quiz = json.loads(quiz_string)

In [45]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

print(f"Number of items in quiz_table_data: {len(quiz_table_data)}")

Number of items in quiz_table_data: 5


In [46]:
quiz_table_data

[{'MCQ': 'A critical design consideration for transmitters, directly impacting signal strength and susceptibility to noise, is:',
  'Choices': 'a: Modulation type | b: Carrier wave frequency | c: Power consumption | d: Antenna size',
  'Correct': 'c'},
 {'MCQ': 'Which statement regarding instrumentation engineering is MOST accurate?',
  'Choices': 'a: It primarily focuses on theoretical physics principles. | b: It designs devices exclusively for standalone measurement tasks. | c: It often utilizes principles like the Doppler effect for measurement. | d: It has minimal overlap with control engineering concepts.',
  'Correct': 'c'},
 {'MCQ': 'The evolution of electronics engineering was significantly propelled by the invention of the:',
  'Choices': 'a: Capacitor | b: Resistor | c: Transistor | d: Mechanical switch',
  'Correct': 'c'},
 {'MCQ': 'VLSI design engineers primarily work with which stage of integrated circuit development?',
  'Choices': 'a: Testing and debugging fabricated ICs

In [47]:
quiz = pd.DataFrame(quiz_table_data)

In [48]:
quiz.to_csv("electronics.csv",index= False)